In [1]:
#Imports

import numpy as np
import matplotlib.pyplot as plt
import hera_cal.abscal as abscal
import uvtools.dspec as dspec 
import hera_pspec
from hera_pspec import pspecbeam
from hera_pspec import pspecdata
import itertools
import scipy 
from scipy import signal
import pickle
import copy
from hera_cal.utils import polnum2str, polstr2num, jnum2str, jstr2num
from hera_cal.io import HERAData, HERACal
from hera_cal.io import DataContainer 
from hera_cal import apply_cal
from hera_cal import io
from hera_cal import smooth_cal
from hera_cal import vis_clean
from hera_cal import redcal
from pyuvdata import UVFlag
import glob
import tqdm
import os
import shutil
from hera_cal import frf
import imp
from hera_pspec import utils as uvp_utils
from hera_pspec import plot as pspec_plot
from hera_pspec import grouping

/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:22: UserWarning: HealVis failed to import.
  warnings.warn("HealVis failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:27: UserWarning: PRISim failed to import.
  warnings.warn("PRISim failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/visibilities/__init__.py:33: UserWarning: VisGPU failed to import.
  warnings.warn("VisGPU failed to import.")
/users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_sim/__init__.py:36: FutureWarning: 
In the next major release, all HERA-specific variables will be removed from the codebase. The following variables will need to be accessed through new class-like structures to be introduced in the next major release: 

noise.HERA_Tsky_mdl
noise.HERA_BEAM_POLY
sigchain.HERA_NRAO_BANDPASS
rfi.HERA_RFI_STATIONS

Additionally, the next maj

In [ ]:
#Load ABF and AAF Combined Data
file_before = 'AverageBeforeFilter/data_avg.Cavg.uvh5'
file_after= 'AverageAfterFilter/data_avg.Cavg.uvh5'

avg_before_filter = HERAData(file_before)
avg_after_filter = HERAData(file_after)

avg_bf_data, avg_bf_flags, avg_bf_nsamples = avg_before_filter.read()
avg_af_data, avg_af_flags, avg_af_nsamples = avg_after_filter.read()


In [ ]:
#Selecting Specific LSTS

selection = (avg_before_filter.lsts*(24/(2*np.pi)) >= 4.2) & (avg_before_filter.lsts*(24/(2*np.pi)) <= 6.2)
avg_before_filter.select(times = avg_before_filter.times[selection])
avg_after_filter.select(times = avg_after_filter.times[selection])

In [ ]:
#Get Matching Redundant Baseline Groups

bls1, bls2, _, _, _, red_groups, red_lens, _ = uvp_utils.calc_blpair_reds(avg_before_filter,avg_after_filter,
                                                                          exclude_auto_bls=True,extra_info=True)


red_antpairs = [[] for m in range(np.max(red_groups)+1)]
for group,bl1,bl2 in zip(red_groups,bls1,bls2):
    red_antpairs[group].append((bl1,bl2))
    
#Redundant Baseline Groups (Specifically Short Baselines from 14m - 45m)
bls1_lim, bls2_lim, _, _, _, red_groups_lim, red_lens_lim, _ = uvp_utils.calc_blpair_reds(avg_before_filter,avg_after_filter,
                                                                          exclude_auto_bls=True,extra_info=True,
                                                                          bl_len_range=(14,45))


red_antpairs_lim = [[] for m in range(np.max(red_groups_lim)+1)]
for group,bl1,bl2 in zip(red_groups_lim,bls1_lim,bls2_lim):
    red_antpairs_lim[group].append((bl1,bl2)) 
    
#Negating 0 Antennapairs/Baseline Groups
red_antpairs = [x for x in red_antpairs if x != []]
red_antpairs_lim = [x for x in red_antpairs_lim if x != []]

In [ ]:
#Power Spectrum Estimation and Conversion

lambd = 3e+8 / 150e+6 
fwhm = (2.44*lambd)/14
airy_beam = pspecbeam.PSpecBeamGauss(fwhm,avg_after_filter.freqs)

psd_before = pspecdata.PSpecData(dsets=[avg_before_filter], beam=airy_beam)
psd_before.Jy_to_mK()
psd_after = pspecdata.PSpecData(dsets=[avg_after_filter], beam=airy_beam)
psd_after.Jy_to_mK()

#Selecting Spectral Windows
psd_before.broadcast_dset_flags(spw_ranges=[(357,562), (20,184)])
psd_after.broadcast_dset_flags(spw_ranges=[(357,562), (20,184)])

#Calculating Power Spectrum of ABF & AAF
uvp_before = psd_before.pspec(bls1=bls1,
                              bls2=bls2,
                              taper='bh7', dsets=[0,0],pols=('yy','yy'),
                              sampling=True,store_window=False,
                              spw_ranges=[(357,562), (20,184)], allow_fft=True)
uvp_before.average_spectra(time_avg=True,blpair_groups=red_antpairs)

uvp_after = psd_after.pspec(bls1=bls1,
                              bls2=bls2,
                              taper='bh7', dsets=[0,0],pols=('yy','yy'),
                              sampling=True,store_window=False,
                              spw_ranges=[(357,562), (20,184)], allow_fft=True)

uvp_after.average_spectra(time_avg=True,blpair_groups=red_antpairs)

#Calculating Power Spectrum of ABF & AAF (Reduced to Short Baselines)
uvp_before_lim = psd_before.pspec(bls1=bls1_lim,
                              bls2=bls2_lim,
                              taper='bh7', dsets=[0,0],pols=('yy','yy'),
                              sampling=True,store_window=False,
                              spw_ranges=[(357,562), (20,184)], allow_fft=True)

uvp_before_lim.average_spectra(time_avg=True,blpair_groups=red_antpairs_lim)

uvp_after_lim = psd_after.pspec(bls1=bls1_lim,
                              bls2=bls2_lim,
                              taper='bh7', dsets=[0,0],pols=('yy','yy'),
                              sampling=True,store_window=False,
                              spw_ranges=[(357,562), (20,184)], allow_fft=True)

uvp_after_lim.average_spectra(time_avg=True,blpair_groups=red_antpairs_lim)

In [ ]:
#Get Radial Cosmological Wavevectors

k_data_before = uvp_before.get_kparas(0)[104::2]
k_bins_before = np.diff(k_data_before)
k_data_after = uvp_after.get_kparas(0)[104::2]
k_bins_after= np.diff(k_data_after)

k_data_before_lim = uvp_before_lim.get_kparas(0)[104::2]
k_bins_before_lim = np.diff(k_data_before_lim)
k_data_after_lim = uvp_after_lim.get_kparas(0)[104::2]
k_bins_after_lim = np.diff(k_data_after_lim)

#Perform a Spherical Average
uvp_before_sph = grouping.spherical_average(uvp_in=uvp_before,kbins=k_data_before[:-1], bin_widths=k_bins_before)
uvp_after_sph = grouping.spherical_average(uvp_in=uvp_after,kbins=k_data_after[:-1], bin_widths=k_bins_after)

uvp_before_sph_lim = grouping.spherical_average(uvp_in=uvp_before_lim,kbins=k_data_before_lim[:-1], bin_widths=k_bins_before_lim)
uvp_after_sph_lim = grouping.spherical_average(uvp_in=uvp_after_lim,kbins=k_data_after_lim[:-1], bin_widths=k_bins_after_lim)
